## Comparing EM with GD

### First, we set up by choosing the number of clusters $m$, the number of features $n$, and generate samples by use of a BMM with random parameters. The goal is to recover this setting. The sample set is $S$, which is a dictionary containing binary strings $x$. Notice that replicates are gathered together to save space.

In [43]:
import nbm
import numpy as np
m = 2  # number of classes
n = 10 # number of features
samples = 2**20 # sample size
# generate the parameters

In [47]:
[gtheta, gPhi] = nbm.randInt(m, n)
total = np.zeros(n)
ct = 0
# generate samples
S = dict()
for _ in range(2**n):    # the sample space
# compute the probability
    x = nbm.decToBits(_, n)   # convert decimal to binary
    prob = 0.0
    for __ in range(m):
        prob += gtheta[__] * nbm.bern_n(x, gPhi[__, :], n) 
    S[_] = prob * samples

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.34 µs


In [29]:
[theta, Phi] = nbm.randInt(m, n)
theta

array([0.497, 0.503])

In [40]:
%prun [ell2, theta, Phi, iterId] = nbm.gradientDescent(n, samples, S, theta, Phi, numIterations=10000, alpha = 0.02, tolerance = 1e-7)

In [ ]:
sum_ratio = 0
count = 0
print("m, n:", m, n)
for __ in range(200):
    [gtheta, gPhi] = nbm.randInt(m, n)
    total = np.zeros(n)
    ct = 0
    # generate samples
    S = dict()
    for _ in range(2**n):    # the sample space
    # compute the probability
        x = nbm.decToBits(_, n)   # convert decimal to binary
        prob = 0.0
        for __ in range(m):
            prob += gtheta[__] * nbm.bern_n(x, gPhi[__, :], n) 
        S[_] = prob * samples
    # EM
    ell0 = nbm.ell(n, gPhi, gtheta, S, samples)
    [theta, Phi] = nbm.randInt(m, n)
    [theta1, Phi1] = nbm.em(n, m, samples, S, theta, Phi)
    ell1 = nbm.ell(n, Phi1, theta1, S, samples)
    # GD
    find_k = 0   # if a k-cluster point is found
    for _ in range(10):
        [theta, Phi] = nbm.randInt(m, n)
        [ell2, theta, Phi, iterId] = nbm.gradientDescent(n, samples, S, theta, Phi, numIterations=10000, alpha = 0.02, tolerance = 1e-7)
        if min(theta) < 0.0005:
            find_k = 1
            break
    if find_k == 1:
        ratio = np.exp(ell1 - ell2)
        print("GD/EM: ", ratio, "count: ", count)
        print("GD/opt: ", np.exp(ell0 - ell2))
        print("EM/opt: ", np.exp(ell0 - ell1))
  #      [rand_theta, rand_Phi] = nbm.randInt(m, n)
  #      ell_rand = nbm.ell(n, rand_Phi, rand_theta, S, samples)
 #       print("rand/opt: ", np.exp(ell0 - ell_rand))
        sum_ratio += ratio 
        count += 1
    if count == 60:    # if collected 60 points
        print("avg ratio: ", sum_ratio / count)
        break

m, n: 6 3
GD/EM:  0.9999998906602485 count:  0
GD/opt:  0.9999999968348123
EM/opt:  1.0000001061745754
GD/EM:  1.0000002165056028 count:  1
GD/opt:  0.999999958484312
EM/opt:  0.9999997419787652
GD/EM:  1.0000002138879993 count:  2
GD/opt:  0.9999999987102222
EM/opt:  0.9999997848222689
GD/EM:  0.9999995486800163 count:  3
GD/opt:  0.9999999999999445
EM/opt:  1.000000451320132
GD/EM:  1.0000000550367585 count:  4
GD/opt:  0.9999999999999636
EM/opt:  0.9999999449632082
GD/EM:  1.0000002845820208 count:  5
GD/opt:  0.9999999999999647
EM/opt:  0.9999997154180248
GD/EM:  1.0000003723898625 count:  6
GD/opt:  0.999999999999909
EM/opt:  0.9999996276101851
GD/EM:  0.9999999612601101 count:  7
GD/opt:  0.999999999997994
EM/opt:  1.0000000387378853
GD/EM:  1.0000001320300607 count:  8
GD/opt:  0.9999999999999163
EM/opt:  0.9999998679698731
GD/EM:  0.9999998279244969 count:  9
GD/opt:  0.9999999999999121
EM/opt:  1.0000001720754448
GD/EM:  1.0000002150180556 count:  10
GD/opt:  0.999999999999941

In [26]:
(m, n)

(5, 3)